In [ ]:
import mysql.connector

# 连接到本地 MySQL 数据库
connection = mysql.connector.connect(
    host="localhost",
    user="root",
    password="123456",
    database="your_database"
)
# 创建游标对象
cursor = connection.cursor()

# 获取所有表名
cursor.execute("SHOW TABLES")

# 遍历所有表
for (table_name,) in cursor:
    # 读取表的定义
    cursor.execute(f"SHOW CREATE TABLE {table_name}")

    # 获取表的定义
    result = cursor.fetchone()

    # 打印表的定义
    print(f"Table: {table_name}")
    print(result[1])  # 第二个元素包含了表的定义
    print()

# 关闭游标和连接
cursor.close()
connection.close()


In [ ]:
import argparse
import time
import json
from openai import APIConnectionError, OpenAI
from progress.bar import Bar
import concurrent.futures
import re

import random

import yaml

def load_config(file_path):
    with open(file_path, 'r') as config_file:
        config = yaml.safe_load(config_file)
    return config

config = load_config('config.yaml')

client = OpenAI(
    api_key=config['api']['key']
)

def makecomversation(prompt, usercontent, return_format, temperature=0):
    def sendquestion(prompt, usercontent):
        message = [
                {
                    "role": "system",
                    "content": prompt
                },
                {
                    "role": "user",  
                    "content": usercontent + " Return in JSON format: " + return_format
                }
            ]
        
        chat_completion = client.chat.completions.create(
            model="gpt-4-1106-preview",
            messages=message,
            temperature=temperature,
            response_format={"type": "json_object"}
            )
        return chat_completion

    try:
       chat_completion = sendquestion(prompt, usercontent)
    except Exception as e: 
       chat_completion = sendquestion(prompt, usercontent)
       
    answer = chat_completion.choices[0].message.content
    return json.loads(answer) 


def nl2sql(question, fewshot = True):
    
    if fewshot:
        with open('spider_examples.txt', 'r', encoding='utf-8') as f:
            spider_examples = f.readlines()
        random_selection = random.sample(spider_examples, 10)
    
    prompt = f"""
    You are a SQLite expert, you are proficient in writing SQLite SOL queries.
            You are given a SQLite database definition and a question list both in JSON format. 
            the database definition json is formated as below:
            [(table_name:, create_sql:), (table_name:, create_sql:),..]
            where you need to understand the definition of tables from create_sql statements.
            then you need to write corresponding SQL query based on the database definition and the question.
            output json format below: 
            [(question : ", sql : ),(question : ", sql : "),..]
            Please output json only, No Descriptions.

            the database definition json is:
            {database}

            the question list is:
            {question}
            
    here is some examples:
    {random_selection}
    """

    usercontent = f""" You need to translate the sql provided and then return coresponding question. 
    Because sometimes SQL is too complex to understand, or even contains errors, we will also provide a simplified version(a clause list, contains main information about the sql) for reference. 
    In addition, we will also provide some tips to introduce the meaning of this sql, and what the generated question is mostly about.
    Do not generate any other content, such as "The question is: ", just the question of the sql itself. 
    if you can't translate the sql, return the simplified version provided(needs to be concat in right order), don't reply content as such "The SQL clause is not valid. Please provide a valid SQL clause.".
    The original sql is: 
    {question}
    """

    return_format = '{"sql":"coresponding sql to the question provided"}'

    result = makecomversation(prompt, usercontent, return_format)

    return result['sql']

In [ ]:
## 并行生成question_list 中的每个问题的sql

question_list = []

sql_question_pairs = []  # 结果
err_list = [] # 生成失败的问题
with concurrent.futures.ThreadPoolExecutor() as executor:
    future_to_sentence = {executor.submit(nl2sql, question): question for question in question_list}
    
    for future in concurrent.futures.as_completed(future_to_sentence):
        question = future_to_sentence[future]
        try:
            sql = future.result()
            sql_question_pairs.append({'sql': question, 'question': sql})
        except Exception as e:
            print("翻译句子",question,"时出现异常:", e)
            err_list.append(question)